In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px

: 

In [ ]:
df = pd.read_csv("C:\Users\Domodekavkaz\AppData\Roaming\Holovibes\12.10.0\benchmark\benchmark_2024-04-22_11h14-07.csv", header=1)

In [ ]:
# print the list of columns
print(df.columns)

In [ ]:
# Afficher les premières lignes du DataFrame
print(df.head())

# Extraire les métadonnées du fichier CSV
with open('kek4_1.csv', 'r') as file:
    first_line = file.readline()
    metadata = first_line.split(',')

print("Métadonnées :")
for meta in metadata:
    print(meta)


In [ ]:
# make a dictionary of the metadata, for each element of metadata separated into two part from ': ', the key is the first part, and the value is the second part

metadata_dict = {}
for meta in metadata:
    key, value = meta.split(': ')
    metadata_dict[key] = value

In [ ]:
# print the list of columns
print(df.index)

In [ ]:
import plotly.express as px

# use plotly to create a line chart that has each "Queue" column as a line, and the x-axis is the length of the index
fig = px.line(df, x=range(len(df)), y=['Input Queue', 'Output Queue', 'Record Queue'])
# add à horizontal blue line at the level of the "GPU input queue size" metadata
fig.add_hline(y=int(metadata_dict['GPU Input Queue size']), line_dash="dot", line_color="blue", annotation_text="GPU input queue size", annotation_position="bottom right")
# add à horizontal red line at the level of the "GPU output queue size" metadata
fig.add_hline(y=int(metadata_dict['GPU Output Queue size']), line_dash="dot", line_color="red", annotation_text="GPU output queue size", annotation_position="bottom right")
# add à horizontal green line at the level of the "GPU record queue size" metadata
fig.add_hline(y=int(metadata_dict['CPU Record Queue size']), line_dash="dot", line_color="green", annotation_text="CPU record queue size", annotation_position="bottom right")

fig.show()